<h2 align="center">Codebasics DL Course: Vehicle Damage Detection Project: Hyperparameter Tunning</h2>

In [1]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time
import torchvision.models as models
from matplotlib import pyplot as plt
import optuna

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

### Load Data

In [3]:
image_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
dataset_path = "./dataset"

dataset = datasets.ImageFolder(root=dataset_path, transform=image_transforms)
len(dataset)

2300

In [5]:
2300*0.75

1725.0

In [6]:
class_names = dataset.classes
class_names 

['F_Breakage', 'F_Crushed', 'F_Normal', 'R_Breakage', 'R_Crushed', 'R_Normal']

In [7]:
num_classes = len(dataset.classes)
num_classes

6

In [8]:
train_size = int(0.75*len(dataset))
val_size = len(dataset) - train_size

train_size, val_size

(1725, 575)

In [9]:
from torch.utils.data import random_split

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [10]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

### Model Training & Hyperparameter Tunning

In [11]:
# Load the pre-trained ResNet model
class CarClassifierResNet(nn.Module):
    def __init__(self, num_classes, dropout_rate=0.5):
        super().__init__()
        self.model = models.resnet50(weights='DEFAULT')
        # Freeze all layers except the final fully connected layer
        for param in self.model.parameters():
            param.requires_grad = False
            
        # Unfreeze layer4 and fc layers
        for param in self.model.layer4.parameters():
            param.requires_grad = True            
            
        # Replace the final fully connected layer
        self.model.fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(self.model.fc.in_features, num_classes)
        )

    def forward(self, x):
        x = self.model(x)
        return x

In [12]:
# Define the objective function for Optuna
def objective(trial):
    # Suggest values for the hyperparameters
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.7)
    
    # Load the model
    model = CarClassifierResNet(num_classes=num_classes, dropout_rate=dropout_rate).to(device)
    
    # Define the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
    
    # Training loop (using fewer epochs for faster hyperparameter tuning)
    epochs = 3
    start = time.time()
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for batch_num, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        
        # Validation loop
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        
        # Report intermediate result to Optuna
        trial.report(accuracy, epoch)
        
        # Handle pruning (if applicable)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    end = time.time()
    print(f"Execution time: {end - start} seconds")
    
    return accuracy

In [13]:
# Create the study and optimize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

[I 2025-07-28 14:40:05,944] A new study created in memory with name: no-name-3b54ce20-141d-4649-bb21-1f9bc6c1901e
[I 2025-07-28 14:46:43,561] Trial 0 finished with value: 77.3913043478261 and parameters: {'lr': 0.00036500442855530815, 'dropout_rate': 0.6275201463555593}. Best is trial 0 with value: 77.3913043478261.


Execution time: 396.77863931655884 seconds


[I 2025-07-28 14:53:05,996] Trial 1 finished with value: 76.8695652173913 and parameters: {'lr': 0.0026998906313090883, 'dropout_rate': 0.4481107402327942}. Best is trial 0 with value: 77.3913043478261.


Execution time: 381.69168734550476 seconds


[I 2025-07-28 14:58:58,790] Trial 2 finished with value: 80.17391304347827 and parameters: {'lr': 0.0007602376961789882, 'dropout_rate': 0.6844221814374176}. Best is trial 2 with value: 80.17391304347827.


...

[I 2025-07-28 15:11:43,328] Trial 4 finished with value: 79.30434782608695 and parameters: {'lr': 0.00017046507196768763, 'dropout_rate': 0.2942246887537656}. Best is trial 3 with value: 80.69565217391305.


Execution time: 396.3323323726654 seconds


[I 2025-07-28 15:13:39,320] Trial 5 pruned. 
[I 2025-07-28 15:15:30,980] Trial 6 pruned. 
[I 2025-07-28 15:19:16,607] Trial 7 pruned. 
[I 2025-07-28 15:21:13,265] Trial 8 pruned. 
[I 2025-07-28 15:23:03,735] Trial 9 pruned. 
[I 2025-07-28 15:25:08,272] Trial 10 pruned. 
[I 2025-07-28 15:29:45,524] Trial 11 pruned. 
[I 2025-07-28 15:36:30,733] Trial 12 pruned. 
[I 2025-07-28 15:42:53,742] Trial 13 pruned. 
[I 2025-07-28 15:44:55,038] Trial 14 pruned. 
[I 2025-07-28 15:47:01,726] Trial 15 pruned. 
[I 2025-07-28 15:49:01,485] Trial 16 pruned. 
[I 2025-07-28 15:56:49,545] Trial 17 finished with value: 80.17391304347827 and parameters: {'lr': 0.001500788008070497, 'dropout_rate': 0.20175540841582965}. Best is trial 3 with value: 80.69565217391305.


Execution time: 467.3426558971405 seconds


[I 2025-07-28 15:59:16,007] Trial 18 pruned. 
[I 2025-07-28 16:03:25,629] Trial 19 pruned. 


In [14]:
study.best_params

{'lr': 0.004789960721721224, 'dropout_rate': 0.3456729697739871}

In the third run it gave me best results with lr=0.005 and dropout=0.3.